## 1. Создание предсказательной модели и оценивание её качества.

Считаем данные из скачанной открытой гугл-папки

In [3]:
import os
import pandas as pd
path = r'C:\Users\pinkm\Downloads\mini-course-public' 
data = {}

Data = []
Target = []

i=0
for dir_entry in sorted(os.listdir(path)):
    dir_entry_path = os.path.join(path, dir_entry)
    if os.path.isfile(dir_entry_path):
        i+=1
        with open(dir_entry_path, 'r') as my_file:
            print(dir_entry_path.split("\\")[-1])
            try:
                df = pd.read_csv(my_file, delimiter=';')
                if df.shape[1] == 5:
                    Data.append(df)
                    Target.append(dir_entry_path.split("\\")[-1])
            except UnicodeDecodeError:
                print(dir_entry_path.split("\\")[-1])
                pass

for dir_entry in sorted(os.listdir(path)):
    dir_entry_path = os.path.join(path, dir_entry)
    if os.path.isfile(dir_entry_path):
        i+=1
        with open(dir_entry_path, 'r') as my_file:
            print(dir_entry_path.split("\\")[-1])
            try:
                df = pd.read_csv(my_file, delimiter=',')
                if df.shape[1] == 5:
                    Data.append(df)
                    Target.append(dir_entry_path.split("\\")[-1])
            except UnicodeDecodeError:
                print(dir_entry_path.split("\\")[-1])
                pass

FileNotFoundError: [WinError 3] Системе не удается найти указанный путь: 'C:\\Users\\pinkm\\Downloads\\mini-course-public'

Переведём типы движений в числа

In [ ]:
classes = {'тояни' : 0, 'месте' : 0, 'одьб' : 1, 'аг' : 1, 'ег' : 2, 'елоси' : 3, 'естн' : 4, 'одъ' : 4, 
           'втомо' : 5, 'ашин' : 5, 'метро' : 6, 'лектро' : 7, 'амок' : 8}

answers = {'стояние' : 0, 'ходьба' : 1, 'бег' : 2, 'велосипед' : 3, 'лестница' : 4, 
           'автомобиль' : 5, 'метро' : 6, 'электросамокат' : 7, 'самокат' : 8}

# стояние, ходьба, велосипед, лестница, бег, подъем, подъём, автомобиль, электросамокат, машина, метро

In [ ]:
def get_action(name):
    for act in classes:
        if name.lower().find(act) != -1:
            return classes[act]
    return -1

Обработаем данные и извлечём признаки для создания модели

In [2]:
import pandas as pd
from matplotlib import pylab as plt
%pylab inline

def process_data(df, comma=True, cut=True, name = "", a=-1, b=-1):
    
# обрезаем начало и конец трека
    if a == -1:
        st = 1250
    else:
        st = a
            
    if b == -1:
        end = -1250
    else:
        end = b
        
    if cut == False:
        st = 0
        end = -1

    print(name)
#     fig, ((ax1, ax2)) = plt.subplots(nrows=1, ncols=2, figsize=(15,6))

# приводим данные с Iphone к нужному формату
    if comma is True:
        df = df.applymap(lambda x: str(x).replace(',','.'))
        
        try:
            df['gFx'] = df['gFx'].astype(float)
            df['gFy'] = df['gFy'].astype(float)
            df['gFz'] = df['gFz'].astype(float)
        except KeyError:
            return -1

# рисуем графики ускорений
#     if cut:
#         ax1.plot(df.iloc[st:end]['gFx'],c='g')
#         ax1.plot(df.iloc[st:end]['gFy'],c='r')
#         ax1.plot(df.iloc[st:end]['gFz'],c='b')
#     else:
#         ax1.plot(df['gFx'],c='g')
#         ax1.plot(df['gFy'],c='r')
#         ax1.plot(df['gFz'],c='b')
        
# вычисляем модуль (норму) ускорения и рисуем его
    Acc = (df.gFx[st:end] ** 2 + df.gFy[st:end] ** 2 + df.gFz[st:end] ** 2) ** 0.5
#     ax2.plot(Acc)
#     plt.show()

# можно различить многие действия по амплитуде ускорения
    amplitude = Acc.max() - Acc.min()
    print(' АМПЛИТУДА:', Acc.max()-Acc.min())
    print()
    
    return amplitude

Ampl = []
for df, target in zip(Data,Target):
    Ampl.append(process_data(df, name=target))

Populating the interactive namespace from numpy and matplotlib


c:\users\pinkm\appdata\local\programs\python\python39\lib\site-packages\IPython\core\magics\pylab.py:159: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


Напишем функцию для получения предсказания (здесь используется только амплитуда, но в вашем итоговом решении будет больше признаков)

In [20]:
def get_prediction(ampl):
    if ampl < 1:
        return 0 #стояние
    if ampl >  5:
        return 2 #бег
    return 1 #ходьба

In [41]:
print(len(Data), len(Ampl), len(Target))

936 936 936


Некоторые амплитуды посчитаны некорректно, уберём соответствующие треки.

In [42]:
good_indices = [i for i in range(len(Ampl)) if not (math.isnan(Ampl[i]) or Ampl[i]<=0)]

GoodData   = [Data[i] for i in good_indices]
GoodAmpl   = [Ampl[i] for i in good_indices]
GoodTarget = [Target[i] for i in good_indices]

In [48]:
len(Data), len(GoodAmpl)

(936, 868)

Создадим таблицу, содержащую номер трека, правильный ответ и предсказание.

In [58]:
prediction = [get_prediction(a) for a in GoodAmpl]
answer = [get_action(target) for target in GoodTarget]

Res = pd.DataFrame({'track_num' : np.arange(len(GoodData)), 'answer' : answer, 'action' : prediction})
Res.head(10)
Res = Res[Res['answer'] > 0]

Проверим качество нашей модели на тренировочных данных (на данных из открытой папки)

In [59]:
from sklearn.metrics import accuracy_score

accuracy_score(Res['answer'], Res['action'])

0.502454991816694

В тренировочной папке количество треков разных типов несбалансировано, поэтому для корректной оценки качества нашей модели будем проверять качество модели на таком же распределении треков, как и в тестовой выборке:

* 18% - стояние
* 18 % - ходьба
* 18 % - бег
* 18 % - подъем по лестнице
* 18 % - велосипед
* 10% - другие типы движений

In [60]:
Res['answer'].value_counts(), len(Res)

(1    224
 2    168
 4    145
 3     40
 5     22
 7      8
 6      4
 Name: answer, dtype: int64,
 611)

In [61]:
import random

ind_0 = random.sample(Res[Res.answer == 0].index.tolist(), 32)
ind_1 = random.sample(Res[Res.answer == 1].index.tolist(), 32)
ind_2 = random.sample(Res[Res.answer == 2].index.tolist(), 32)
ind_3 = random.sample(Res[Res.answer == 3].index.tolist(), 32)
ind_4 = random.sample(Res[Res.answer == 4].index.tolist(), 32)
ind_5 = random.sample(Res[Res.answer == 5].index.tolist(), 6)
ind_6 = random.sample(Res[Res.answer == 6].index.tolist(), 6)
ind_7 = random.sample(Res[Res.answer == 7].index.tolist(), 6)

ind_balanced = ind_0 + ind_1 + ind_2 + ind_3 + ind_4 + ind_5 + ind_6 + ind_7 
len(ind_balanced)

ValueError: Sample larger than population or is negative

In [28]:
Res_balanced = Res.iloc[ind_balanced]

NameError: name 'ind_balanced' is not defined

In [29]:
accuracy_score(Res_balanced['answer'], Res_balanced['action'])

NameError: name 'Res_balanced' is not defined